<a href="https://colab.research.google.com/github/mohamedfadul/DeepSearch-Finding-Truncated-Differential-Trails-Using-Genetic-Algorithms-and-Deep-Learning/blob/main/ANN_Baseline_Model_to_Predict_Trail_Validity_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [4]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [5]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

print(tf.__version__)

2.12.0


In [6]:
!wget -O all_dataset_path.csv "https://drive.google.com/uc?export=download&id=1vG2C0PM3MXz7p_v5zMSjvodRsK6hbrnz"

--2023-07-11 09:09:51--  https://drive.google.com/uc?export=download&id=1vG2C0PM3MXz7p_v5zMSjvodRsK6hbrnz
Resolving drive.google.com (drive.google.com)... 209.85.147.102, 209.85.147.139, 209.85.147.100, ...
Connecting to drive.google.com (drive.google.com)|209.85.147.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nqkqvd26fh5rvnjilq6d1dc3rfpfvf1e/1689066525000/14347683118445347781/*/1vG2C0PM3MXz7p_v5zMSjvodRsK6hbrnz?e=download&uuid=b5957833-d358-405d-81d9-e196299141d7 [following]
--2023-07-11 09:09:52--  https://doc-08-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nqkqvd26fh5rvnjilq6d1dc3rfpfvf1e/1689066525000/14347683118445347781/*/1vG2C0PM3MXz7p_v5zMSjvodRsK6hbrnz?e=download&uuid=b5957833-d358-405d-81d9-e196299141d7
Resolving doc-08-54-docs.googleusercontent.com (doc-08-54-docs.googleusercontent.com)... 173.194.193.132, 2607

In [7]:
all_dataset = pd.read_csv ('all_dataset_path.csv')

In [8]:
all_dataset

,input16,input15,input14,input13,input12,input11,input10,input9,input8,input7,...,perm7,perm8,perm9,perm10,perm11,perm12,perm13,perm14,perm15,perm16
0,1,1,1,1,1,1,1,1,1,1,...,0.200000,0.533333,0.866667,0.4,0.600000,0.133333,1.0,0.666667,0.733333,0.933333
1,1,0,1,1,1,1,1,1,1,1,...,0.200000,0.533333,0.866667,0.4,0.600000,0.133333,1.0,0.666667,0.733333,0.933333
2,1,1,1,1,1,1,1,1,1,1,...,0.200000,0.533333,0.866667,0.4,0.600000,0.133333,1.0,0.666667,0.733333,0.933333
3,1,1,1,1,1,1,0,1,1,1,...,0.200000,0.533333,0.866667,0.4,0.600000,0.133333,1.0,0.666667,0.733333,0.933333
4,1,1,0,1,1,1,1,1,1,1,...,0.200000,0.533333,0.866667,0.4,0.600000,0.133333,1.0,0.666667,0.733333,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1,1,1,1,1,1,1,1,1,0,...,0.466667,0.533333,1.000000,0.8,0.333333,0.933333,0.6,0.000000,0.866667,0.666667
11996,1,1,1,1,1,1,1,1,1,0,...,0.466667,0.533333,1.000000,0.8,0.333333,0.933333,0.6,0.000000,0.866667,0.666667
11997,1,1,1,1,1,1,1,1,1,1,...,0.466667,0.533333,1.000000,0.8,0.333333,0.933333,0.6,0.000000,0.866667,0.666667
11998,1,1,1,1,1,1,1,1,1,1,...,0.466667,0.533333,1.000000,0.8,0.333333,0.933333,0.6,0.000000,0.866667,0.666667


In [9]:
dataset_before_undersampling = all_dataset.sample(frac=0.80,random_state=0)
test_dataset = all_dataset.drop(dataset_before_undersampling.index)

In [10]:
dataset_before_undersampling['sboxes'] = ((dataset_before_undersampling['sboxes']/(dataset_before_undersampling['round']*8)))

In [11]:
test_dataset['sboxes'] = ((test_dataset['sboxes']/(test_dataset['round']*8)))

In [12]:
# Class count
count_class_0, count_class_1 = dataset_before_undersampling.validity.value_counts()

# Divide by class
df_class_0 = dataset_before_undersampling[dataset_before_undersampling['validity'] == 0]
df_class_1 = dataset_before_undersampling[dataset_before_undersampling['validity'] == 1]

In [13]:
df_class_0.shape

(8741, 51)

In [14]:
df_class_1.shape

(859, 51)

In [15]:
#UnderSampling
# Undersample 0-class and concat the DataFrames of both class
df_class_0_under = df_class_0.sample(count_class_1)
train_dataset = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(train_dataset.validity.value_counts())

Random under-sampling:
0    859
1    859
Name: validity, dtype: int64


In [16]:
train_labels = train_dataset.pop("validity")
test_labels = test_dataset.pop("validity")

In [17]:
train_dataset['round'] = (train_dataset['round'])/100

In [18]:
test_dataset['round'] = (test_dataset['round'])/100

In [19]:
from keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def root_mean_squared_error(y_true, y_pred):
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = math_ops.cast(y_true, y_pred.dtype)
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [20]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [21]:
from keras.layers import Dropout
from keras import regularizers
from keras.regularizers import l2, l1
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

reg_param=0.001
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        layers.Dense(512, activation='relu' , input_shape=[50]),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
      layers.Dense(1,activation='sigmoid' )
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    early_stop = keras.callbacks.EarlyStopping(monitor='accuracy', patience=10,restore_best_weights=True)
    if weights == -1:
        model.fit(X_train, y_train, epochs=50, callbacks=[early_stop])
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return model

In [22]:
model = ANN(train_dataset, train_labels, test_dataset, test_labels, tf.keras.losses.BinaryCrossentropy(), -1)

Epoch 1/50
54/54 [==============================] - 3s 21ms/step - loss: 0.4913 - accuracy: 0.7596
Epoch 2/50
54/54 [==============================] - 1s 15ms/step - loss: 0.2063 - accuracy: 0.9203
Epoch 3/50
54/54 [==============================] - 1s 19ms/step - loss: 0.2191 - accuracy: 0.9249
Epoch 4/50
54/54 [==============================] - 1s 15ms/step - loss: 0.1456 - accuracy: 0.9511
Epoch 5/50
54/54 [==============================] - 1s 16ms/step - loss: 0.1665 - accuracy: 0.9406
Epoch 6/50
54/54 [==============================] - 1s 15ms/step - loss: 0.1360 - accuracy: 0.9470
Epoch 7/50
54/54 [==============================] - 1s 16ms/step - loss: 0.1136 - accuracy: 0.9593
Epoch 8/50
54/54 [==============================] - 1s 20ms/step - loss: 0.1265 - accuracy: 0.9610
Epoch 9/50
54/54 [==============================] - 2s 29ms/step - loss: 0.1527 - accuracy: 0.9453
Epoch 10/50
54/54 [==============================] - 2s 33ms/step - loss: 0.1462 - accuracy: 0.9505
Epoch 11/

In [23]:
model.save('/content/drive/My Drive/Colab Notebooks/Truncated path validity/truncated_validity_model.h5')